In [1]:
!pip install fair-esm

In [ ]:
!pip install tensorflow

In [2]:
!git clone https://github.com/facebookresearch/esm

fatal: destination path 'esm' already exists and is not an empty directory.


In [ ]:
import os
import torch
import sys
sys.path.append('../../../learnMSA')
from learnMSA import msa_hmm
import numpy as np
import zipfile

In [ ]:
!mkdir -p pfam
data_path = "../../../PFAM/no_gaps"
for file in os.listdir(data_path)[-300:]:
    
    if file.endswith(".fasta"):
        family = ".".join(file.split(".")[:-1])
        
        if os.path.exists(f"pfam/{family}.npy"):
            continue
            
        if not os.path.exists(f"pfam/{family}"):
            #generates embeddings if they do not exist so far
            !python esm/scripts/extract.py esm2_t36_3B_UR50D ../../../PFAM/no_gaps/{family}.fasta pfam/{family} --include per_tok >> log.txt
            
        #concatenates all embeddings to numpy arrays and stores them family-wise in a single file
        fasta_file = msa_hmm.fasta.Fasta(data_path+"/"+file, single_seq_ok=True)
        try:
            embeddings = []
            for seqid in fasta_file.seq_ids:
                emb = torch.load(f"pfam/{family}/{seqid}.pt")
                emb_np = emb["representations"][36].detach().numpy()
                embeddings.append(emb_np)

            #concatenate all sequences together in the order of the fasta file
            #we can get them back with that same order and their lengths
            embeddings = np.concatenate(embeddings, 0)    
            np.save(f"pfam/{family}.npy", embeddings)
        
            #remove the torch embeddings (too many single files)
            !rm -r pfam/{family}
        except FileNotFoundError:
            print(f"Embeddings for family {family} are missing. Skipping this family")

In [23]:
#test
data_path = "../../../PFAM/no_gaps"
files = ["PF10249.fasta", "PF06388.fasta"]
for file in files:
    family = ".".join(file.split(".")[:-1])
    embeddings = np.load(f"pfam/{family}.npy")
    fasta_file = msa_hmm.fasta.Fasta(data_path+"/"+file)
    len_sum = 0
    for seqid, length in zip(fasta_file.seq_ids, fasta_file.seq_lens):
        emb = torch.load(f"pfam/{family}/{seqid}.pt")
        emb_np = emb["representations"][36].detach().numpy()
        assert np.all(emb_np == embeddings[len_sum : len_sum + length])
        len_sum += length

In [1]:
!rm -r ~/.cache